# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f86b4b59430>
├── 'a' --> tensor([[ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306]])
└── 'x' --> <FastTreeValue 0x7f86b4b59400>
    └── 'c' --> tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                        [-0.7856, -1.4720, -1.2531,  0.0459],
                        [-1.2467, -2.0849, -1.1003, -0.7238]])

In [4]:
t.a

tensor([[ 0.1991, -2.0661,  0.6654],
        [-0.0267, -0.0546,  0.4306]])

In [5]:
%timeit t.a

70.2 ns ± 1.29 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f86b4b59430>
├── 'a' --> tensor([[ 0.2034, -0.0944, -1.2113],
│                   [-1.0583,  0.2870, -0.6627]])
└── 'x' --> <FastTreeValue 0x7f86b4b59400>
    └── 'c' --> tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                        [-0.7856, -1.4720, -1.2531,  0.0459],
                        [-1.2467, -2.0849, -1.1003, -0.7238]])

In [7]:
%timeit t.a = new_value

78.6 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1991, -2.0661,  0.6654],
               [-0.0267, -0.0546,  0.4306]]),
    x: Batch(
           c: tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1991, -2.0661,  0.6654],
        [-0.0267, -0.0546,  0.4306]])

In [11]:
%timeit b.a

70.3 ns ± 0.769 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4906,  0.9064,  0.8922],
               [ 1.8619,  1.1418, -0.1930]]),
    x: Batch(
           c: tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238]]),
       ),
)

In [13]:
%timeit b.a = new_value

627 ns ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.3 µs ± 68.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

289 µs ± 9.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

264 µs ± 7.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f85feea2d30>
├── 'a' --> tensor([[[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]],
│           
│                   [[ 0.1991, -2.0661,  0.6654],
│                    [-0.0267, -0.0546,  0.4306]]])
└── 'x' --> <FastTreeValue 0x7f85fef18490>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.9 µs ± 726 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f86b4b59b50>
├── 'a' --> tensor([[ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306],
│                   [ 0.1991, -2.0661,  0.6654],
│                   [-0.0267, -0.0546,  0.4306]])
└── 'x' --> <FastTreeValue 0x7f86b4b59b80>
    └── 'c' --> tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                        [-0.7856, -1.4720, -1.2531,  0.0459],
                 

In [23]:
%timeit t_cat(trees)

42.7 µs ± 481 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79 µs ± 641 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0750,  0.8875,  2.9655, -0.0261],
                       [-0.7856, -1.4720, -1.2531,  0.0459],
                       [-1.2467, -2.0849, -1.1003, -0.7238]],
              
                      [[ 0.0750,  0.8875,  2.9655, -0.0261],
                       [-0.7856, -1.4720, -1.2531,  0.0459],
                       [-1.2467, -2.0849, -1.1003, -0.7238]],
              
                      [[ 0.0750,  0.8875,  2.9655, -0.0261],
                       [-0.7856, -1.4720, -1.2531,  0.0459],
                       [-1.2467, -2.0849, -1.1003, -0.7238]],
              
                      [[ 0.0750,  0.8875,  2.9655, -0.0261],
                       [-0.7856, -1.4720, -1.2531,  0.0459],
                       [-1.2467, -2.0849, -1.1003, -0.7238]],
              
                      [[ 0.0750,  0.8875,  2.9655, -0.0261],
                       [-0.7856, -1.4720, -1.2531,  0.0459],
                       [-1.2467, -2.0849, -1.1003, -0.7238]],

In [26]:
%timeit Batch.stack(batches)

105 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238],
                      [ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238],
                      [ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238],
                      [ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238],
                      [ 0.0750,  0.8875,  2.9655, -0.0261],
                      [-0.7856, -1.4720, -1.2531,  0.0459],
                      [-1.2467, -2.0849, -1.1003, -0.7238],
                      [ 0.0750,  0.8875,  2.9655, -0.0261],
                   

In [28]:
%timeit Batch.cat(batches)

190 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

585 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
